In [1]:
import matplotlib.pyplot as plt 
import metapack as mp
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

from metapack.appurl import SearchUrl 
SearchUrl.initialize()

In [2]:
pkg = mp.jupyter.open_source_package()
pkg

In [3]:
r =  pkg.reference('la_jolla_tide')
r

Header,Type,Description
date_time,datetime,
water_level,number,"Water level, relative to datum, in feet"
type,string,"HH Daily high, H High, LL Daily Low, L Low"
I,integer,
L,integer,


In [4]:
water_levels = r.read_csv(parse_dates=True)
wl_by_day = water_levels.set_index(['date_time','type']).water_level.unstack(1).resample('1d').max()
wl_by_day.columns = ['wl_high_1', 'wl_high_2', 'wl_low_1','wl_low_2']
wl_by_day['wl_max'] = np.maximum(wl_by_day.wl_high_1, wl_by_day.wl_high_2 )
wl_by_day['wl_min'] = np.maximum(wl_by_day.wl_low_1, wl_by_day.wl_low_2 )
wl_by_day = wl_by_day[['wl_max','wl_min']]
wl_by_day.head()

,wl_max,wl_max
date_time,,
2000-01-01,5.043,5.043
2000-01-02,4.974,4.974
2000-01-03,5.234,5.234
2000-01-04,5.270,5.270
2000-01-05,5.617,5.617


In [5]:
dist = pkg.reference('sd_river_discharge').read_csv(parse_dates=True).set_index('datetime').resample('1d').mean()
dist = dist[['discharge_rate']].copy()
dist.head()

,discharge_rate
datetime,
1988-01-01,22.0
1988-01-02,20.0
1988-01-03,18.0
1988-01-04,18.0
1988-01-05,21.0


In [6]:
usecols = ['date', 'reporttpye', 'dailyprecip', 'hourlyprecip']
lcd = pkg.reference('local_climate').read_csv(parse_dates=True, usecols=usecols) #.set_index('datetime').resample('1d').mean()

lcd.rename(columns={'reporttpye': 'reporttype'}, inplace=True)
lcd = lcd[lcd.reporttype=='FM-15'].copy()
lcd.fillna(0, inplace=True)
lcd['date'] = lcd.date.dt.date
lcd = lcd.groupby('date').dailyprecip.max()


In [7]:
comb = dist.join(lcd).join(wl_by_day)
comb.head()

,discharge_rate,dailyprecip,wl_max,wl_max
datetime,,,,
1988-01-01,22.0,NaN,NaN,NaN
1988-01-02,20.0,NaN,NaN,NaN
1988-01-03,18.0,NaN,NaN,NaN
1988-01-04,18.0,NaN,NaN,NaN
1988-01-05,21.0,NaN,NaN,NaN


In [8]:
comb.corr()

,discharge_rate,dailyprecip,wl_max,wl_max
discharge_rate,1.000000,0.637036,0.045262,0.045262
dailyprecip,0.637036,1.000000,0.050992,0.050992
wl_max,0.045262,0.050992,1.000000,1.000000
wl_max,0.045262,0.050992,1.000000,1.000000
